In [1]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois
import numpy as np

In [2]:
p = 6661
n = 3
f = poppy.field( p, n )
f.X, f.INV

(Array([[[   1,    0,    0],
         [   0,    1,    0],
         [   0,    0,    1]],
 
        [[   0,    1,    0],
         [   0,    0,    1],
         [   6, 6660,    0]],
 
        [[   0,    0,    1],
         [   6, 6660,    0],
         [   0,    6, 6660]]], dtype=int64),
 Array([   0,    1, 3331, ..., 2220, 3330, 6660], dtype=int64))

In [3]:
p = 13
n = 2
f = poppy.field( p, n )
d = 3
a = poppy.random( ( d, d ), f, poppy.SEED + 0 )
b = poppy.random( ( d, d ), f, poppy.SEED + 1 )

In [4]:
ai = a.inv( )
bi = b.inv( )

In [5]:
print( a.lift[ 0, : 10 ] )
( a - a @ ai @ a @ bi @ b ).lift

[ 1  5 11 10  9  2]


Array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [6]:
p = 31
n = 2
d = 1000
f = poppy.field( p, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ), ( d, d ), 0, f.q, dtype = jnp.int64 )
ap = poppy.array( a, f )
ap.trace( ).proj( )

Array([[244]], dtype=int64)

In [7]:
GF = galois.GF( p, n )
np.trace( GF( np.array( a ) ) )

GF(244, order=31^2)

In [8]:
ap.lift, poppy.v2i( jnp.sum( jnp.pad( ap.lift.ravel( ), ( 0, d*n ) ).reshape( ( -1, d * n * n + n ) )[ : , : n ], axis = 0 ) % p, f )
%timeit poppy.v2i( jnp.sum( jnp.pad( ap.lift.ravel( ), ( 0, d*n ) ).reshape( ( -1, d * n * n + n ) )[ : , : n ], axis = 0 ) % p, f ).block_until_ready( )

33.1 ms ± 3.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit ap.trace( ).lift.block_until_ready( )

30.9 ms ± 512 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit np.trace( GF( np.array( a ) ) )

5.4 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
N  = 10000
d  = 2
p  = 7
p1 = 13
p2 = 17
n  = 6
q  = p ** n
f  = poppy.field( p,  n )
F  = poppy.field( p1, d )
FF = poppy.field( p2, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( N, F.n, F.n ), 0,       q, dtype = jnp.int64 )
AA  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
BB  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
A  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
B  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )

@jax.jit
def poppyhash( a ):
    b = poppy.proj( AA @ a + BB, F )
    c = poppy.proj( A @ b + B, FF )
    return c % ( 4 * N )

liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash )

ap = liftbatch( a, f )
h  = hashbatch( ap ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu ), F.q, FF.q

(10000, 8908, 169, 24137569)